Create LSTM model using dyanamic Data

In [1]:
import pandas as pd
import numpy as np

import yfinance as yf


import warnings 
warnings.filterwarnings('ignore')

Cleaning Data and Extracting usefull features

In [2]:
import yfinance
# dropping nan rows